## <p style="text-align: center"><b>Laaraib Ahmed
### <p style="text-align: center">Nust Student CS
#### <p style="text-align: center;">The following are the Web Scraping implementations as described in the sent mail.

In [114]:

import requests
from bs4 import BeautifulSoup
import re
#Make a dictionary with classes for each attribute
property_dict = {"dict_prices" : [],"dict_address" : [],"dict_sqft" :[],"dict_acres" : [],"dict_url" : [],"dict_broker" : [],}


def fetch_url(url):
    # Headers are used to make the request look like it is coming from a browser.
    HEADERS = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
    }
    # Send an HTTP request to the URL.
    response = requests.get(url, headers = HEADERS)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code:{response.status_code}")
        return None
    
soup=fetch_url("https://www.realtor.com/realestateandhomes-search/Brownwood_TX")    
total_strings=soup.find('div',{"data-testid":"total-matching-properties"}).get_text(strip=True)
total_elements=re.findall(r'\d+',total_strings)[0]
total_elements=int(total_elements)/len((soup.find('section',class_="PropertiesList_propertiesContainer__HTNbx PropertiesList_listViewGrid__U_BlK").find_all('div',class_='BasePropertyCard_propertyCardWrap__Z5y4p')))
print(f"Total Pages of consisting of data are: {int(total_elements)}")

pgnumber=1
while(pgnumber!=int(total_elements)+1):
    # URL of the page we want to scrape.
    soup=fetch_url(f"https://www.realtor.com/realestateandhomes-search/Brownwood_TX/pg-{pgnumber}")
    
    if soup != None:
        if soup.find('div', class_='BasePropertyCard_propertyCardWrap__Z5y4p') == None:
            break    

        price=[price_divs for price_divs in soup.find_all('div', class_='Pricestyles__StyledPrice-rui__btk3ge-0 bvgLFe card-price')]
        property_dict["dict_prices"].append('N/A' if len(price)==0 else [eachprice.get_text(strip=True) for eachprice in price])
        
        address_line_1=([ eachaddress_div.find('div', {'data-testid': 'card-address-1'}) for eachaddress_div in soup.find_all('div', class_='card-address truncate-line')])
        address_line_1=['N/A' if eachaddress_div is None else eachaddress_div.get_text() for eachaddress_div in address_line_1]
        address_line_2=([ eachaddress_div.find('div', {'data-testid': 'card-address-2'}) for eachaddress_div in soup.find_all('div', class_='card-address truncate-line')])
        address_line_2=['N/A' if eachaddress_div is None else eachaddress_div.get_text() for eachaddress_div in address_line_2]
        address_line=[f"{address_line_1[count_element]},{address_line_2[count_element]}" for count_element in range(len(address_line_1))]
        property_dict["dict_address"].append(address_line)

        property_dict["dict_url"].append(['N/A' if url_div is None else f"https://www.realtor.com{url_div.find('a', class_='LinkComponent_anchor__0C2xC').get('href')}"  for url_div in soup.find_all('div', class_='CardContent__StyledCardContent-rui__sc-7ptz1z-0 dsJFdl card-content card-content')])

        property_dict["dict_broker"].append(['N/A' if span_tag is None else span_tag.get_text(strip=True) for span_tag in soup.find_all('span', class_='BrokerTitle_titleText__Y8pb0')])
            
        sqft_li=[property_div for property_div in soup.find_all('ul', class_='PropertyMetastyles__StyledPropertyMeta-rui__sc-1g5rdjn-0 KKDDp card-meta')]
        sqft_li=[None if eachsqft_li is None else eachsqft_li.find('li', class_="PropertySqftMetastyles__StyledPropertySqftMeta-rui__sc-1gdau7i-0 fnhaOV") for eachsqft_li in sqft_li]
        sqft_li=[None if eachsqft_li is None else eachsqft_li.find('span', {"aria-hidden":"true"}) for eachsqft_li in sqft_li]
        property_dict['dict_sqft'].append([0 if eachsqft_li is None else eachsqft_li.find('span', class_='meta-value').get_text(strip=True) for eachsqft_li in sqft_li])
        
        acres=[None if property_div is None else property_div.find('li', class_='PropertyLotSizeMetastyles__StyledPropertyLotSizeMeta-rui__sc-1cz4zco-0 cNMyen') for property_div in soup.find_all('ul', class_='PropertyMetastyles__StyledPropertyMeta-rui__sc-1g5rdjn-0 KKDDp card-meta')]
        property_dict['dict_acres'].append([0 if property_div is None else property_div.find('span', class_='meta-value').get_text(strip=True) for property_div in acres])
         
        print(f"gathered data for {pgnumber} element\n")
        pgnumber += 1
    else:
        break
print("end of fetching data")

Total Pages of consisting of data are: 8
gathered data for 1 element

gathered data for 2 element

gathered data for 3 element

gathered data for 4 element

gathered data for 5 element

gathered data for 6 element

gathered data for 7 element

gathered data for 8 element

end of fetching data


In [122]:
for key,_ in property_dict.items():
    property_dict[key]=[x1 for x in property_dict[key] for x1 in x]

In [124]:
print(len(property_dict["dict_prices"]))
print(len(property_dict["dict_address"]))
print(len(property_dict["dict_url"]))
print(len(property_dict["dict_url"]))
print(len(property_dict["dict_broker"]))
print(len(property_dict["dict_sqft"]))
print(len(property_dict["dict_acres"]))

496
496
496
496
496
496
496


In [125]:
import pandas as pd
df=pd.DataFrame(property_dict)
print((df.head(3)))

  dict_prices                         dict_address dict_sqft dict_acres  \
0     $59,990      Willie B Dr,Brownwood, TX 76801         0       0.31   
1     $55,000  2911 Austin Ave,Brownwood, TX 76801         0          0   
2     $35,000     Long View Dr,Brownwood, TX 76801         0       0.29   

                                            dict_url  \
0  https://www.realtor.com/realestateandhomes-det...   
1  https://www.realtor.com/realestateandhomes-det...   
2  https://www.realtor.com/realestateandhomes-det...   

                                 dict_broker  
0                      Brokered by EG Realty  
1  Brokered by Houston Interests Real Estate  
2                     Brokered by KW Synergy  


In [126]:
with open ('FinalData.csv', 'x') as f:
    df.to_csv(f, header=True, index=False)